In [8]:
import os
import libsql
from dotenv import load_dotenv

load_dotenv()

url = os.getenv("CLAUDELIGHT_DB_URL")
auth_token = os.getenv("CLAUDELIGHT_RW")
conn = libsql.connect("claude_light.db", sync_url=url, auth_token=auth_token)
conn.sync()

# Create Table in db
conn.execute('''create table if not exists
measurements (
id INTEGER PRIMARY KEY, -- this is the rowid
data JSON -- JSON blob (stored as TEXT under the hood)
)''')
conn.sync()

## Collect data to start

In [9]:
# Collect data

import time
import requests
import getpass
import socket
import inspect
import hashlib
import ast

def rgb(R=0, G=0, B=0, tag=None):
    """Run Claude-Light.
    R: float
    The red channel setting (0-1)
    G: float
    The green channel setting (0-1)
    B: float
    The blue channel setting (0-1)
    tag: string
    A label for grouping measurements
    Returns
    -------
    A dictionary with metadata and results
    """
    source = inspect.getsource(rgb)
    # Parse and unparse to normalize formatting
    tree = ast.parse(source)
    normalized = ast.unparse(tree)
    d = {'version': hashlib.sha256(normalized.encode()).hexdigest(), 
         'func': normalized,
         't0': time.time(),
         'user': getpass.getuser(),
         'hostname': socket.gethostname(),
         'tag': tag}
    res = requests.get('https://claude-light.cheme.cmu.edu/api', params={'R': R, 'G': G, 'B': B})
    d.update(res.json())
    return d

In [10]:
# Collect one backgroud data with all 0

res = rgb(tag='background')
res

{'version': 'f5ab4a210c9e404eedc76fed681759d8a800399d82380f4c33d7897ba207226c',
 'func': 'def rgb(R=0, G=0, B=0, tag=None):\n    """Run Claude-Light.\n    R: float\n    The red channel setting (0-1)\n    G: float\n    The green channel setting (0-1)\n    B: float\n    The blue channel setting (0-1)\n    tag: string\n    A label for grouping measurements\n    Returns\n    -------\n    A dictionary with metadata and results\n    """\n    source = inspect.getsource(rgb)\n    tree = ast.parse(source)\n    normalized = ast.unparse(tree)\n    d = {\'version\': hashlib.sha256(normalized.encode()).hexdigest(), \'func\': normalized, \'t0\': time.time(), \'user\': getpass.getuser(), \'hostname\': socket.gethostname(), \'tag\': tag}\n    res = requests.get(\'https://claude-light.cheme.cmu.edu/api\', params={\'R\': R, \'G\': G, \'B\': B})\n    d.update(res.json())\n    return d',
 't0': 1754070252.9737647,
 'user': '26070',
 'hostname': 'Baosen-PC',
 'tag': 'background',
 'in': [0.0, 0.0, 0.0],
 '

In [11]:
# Store data to db

import json
conn.execute("INSERT INTO measurements(data) VALUES (?);", [json.dumps(res)])
conn.commit()
conn.sync()

In [12]:
# Read data from db

for rowid, data in conn.execute("""select * from measurements where json_extract(data, '$.tag') = 'background'""").fetchall():
    j = json.loads(data)
    print(j['out'])

{'415nm': 471, '445nm': 844, '480nm': 1108, '515nm': 1689, '555nm': 1890, '590nm': 2073, '630nm': 2127, '680nm': 2417, 'clear': 13266, 'nir': 5343}
{'415nm': 583, '445nm': 1019, '480nm': 1255, '515nm': 1893, '555nm': 2081, '590nm': 2252, '630nm': 2197, '680nm': 2586, 'clear': 16726, 'nir': 9400}
{'415nm': 401, '445nm': 719, '480nm': 925, '515nm': 1332, '555nm': 1440, '590nm': 1564, '630nm': 1528, '680nm': 1721, 'clear': 9881, 'nir': 3938}


In [13]:
j

{'version': 'f5ab4a210c9e404eedc76fed681759d8a800399d82380f4c33d7897ba207226c',
 'func': 'def rgb(R=0, G=0, B=0, tag=None):\n    """Run Claude-Light.\n    R: float\n    The red channel setting (0-1)\n    G: float\n    The green channel setting (0-1)\n    B: float\n    The blue channel setting (0-1)\n    tag: string\n    A label for grouping measurements\n    Returns\n    -------\n    A dictionary with metadata and results\n    """\n    source = inspect.getsource(rgb)\n    tree = ast.parse(source)\n    normalized = ast.unparse(tree)\n    d = {\'version\': hashlib.sha256(normalized.encode()).hexdigest(), \'func\': normalized, \'t0\': time.time(), \'user\': getpass.getuser(), \'hostname\': socket.gethostname(), \'tag\': tag}\n    res = requests.get(\'https://claude-light.cheme.cmu.edu/api\', params={\'R\': R, \'G\': G, \'B\': B})\n    d.update(res.json())\n    return d',
 't0': 1754070252.9737647,
 'user': '26070',
 'hostname': 'Baosen-PC',
 'tag': 'background',
 'in': [0.0, 0.0, 0.0],
 '

## Collect more data

In [14]:
today_str = time.strftime("%Y%m%d")
print(today_str)

20250801


In [17]:
# Collect more data
import random

for _ in range(10):
    r = round(random.random(), 1)
    g = round(random.random(), 1)
    b = round(random.random(), 1)
    res = rgb(r, g, b, tag=today_str)
    # Store to db
    conn.execute("INSERT INTO measurements(data) VALUES (?);", [json.dumps(res)])
    conn.commit()
    conn.sync()
    
    time.sleep(10)